In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

pd.set_option('display.max_columns', None)

/Users/patrickfahy99/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
sf.set_api_key(api_key='free')
sf.set_data_dir('~/SimFin/simfin_data/')

In [14]:
%%time
# Data for USA.
market = 'us'

# TTM Income Statements.
df_income_ttm = sf.load_income(variant='ttm', market=market)

# Quarterly Income Statements.
df_income_qrt = sf.load_income(variant='quarterly', market=market)

# TTM Balance Sheets.
df_balance_ttm = sf.load_balance(variant='ttm', market=market)

# Quarterly Balance Sheets.
df_balance_qrt = sf.load_balance(variant='quarterly', market=market)

# TTM Cash-Flow Statements.
df_cashflow_ttm = sf.load_cashflow(variant='ttm', market=market)

# Quarterly Cash-Flow Statements.
df_cashflow_qrt = sf.load_cashflow(variant='quarterly', market=market)

# Daily Share-Prices.
df_prices = sf.load_shareprices(variant='daily', market=market)

Dataset "us-income-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-income-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!
CPU times: user 13.9 s, sys: 1.36 s, total: 15.2 s
Wall time: 15.5 s


In [15]:
df_npm = df_income_ttm[NET_INCOME] / df_income_ttm[REVENUE]
df_npm

Ticker  Report Date
A       2010-07-31     0.082820
        2010-10-31     0.125643
        2011-01-31     0.138783
        2011-04-30     0.144574
        2011-07-31     0.157048
                         ...   
low     2018-05-31     0.055455
        2018-08-31     0.055792
        2018-11-30     0.051871
        2019-02-28     0.032450
        2019-05-31     0.033087
Length: 52672, dtype: float64

In [16]:
# Function for calculating growth for a single company.
sales_growth = lambda df_grp: df_grp / df_grp.shift(4) - 1

# Split the DataFrame into sub-groups for the tickers,
# then apply the sales-growth function to each group,
# and finally glue the results back together.
df_growth = df_income_qrt[REVENUE].groupby(TICKER).apply(sales_growth)

# Show the result.
df_growth

Ticker  Report Date
A       2014-01-31          NaN
        2014-04-30          NaN
        2014-07-31          NaN
        2014-10-31          NaN
        2015-01-31     0.017857
                         ...   
low     2018-05-31     0.029656
        2018-08-31     0.071454
        2018-11-30     0.038462
        2019-02-28     0.009810
        2019-05-31     0.021947
Name: Revenue, Length: 50672, dtype: float64

In [17]:
# Calculate 1-year sales-growth.
df_growth = sf.rel_change(df=df_income_qrt[REVENUE], freq='q',
                          years=1, future=False)

# Show the result.
df_growth

Ticker  Report Date
A       2014-01-31          NaN
        2014-04-30          NaN
        2014-07-31          NaN
        2014-10-31          NaN
        2015-01-31     0.017857
                         ...   
low     2018-05-31     0.029656
        2018-08-31     0.071454
        2018-11-30     0.038462
        2019-02-28     0.009810
        2019-05-31     0.021947
Name: Revenue, Length: 50672, dtype: float64

In [18]:
# Data from Income Statements.
df1 = df_income_ttm[NET_INCOME]

# Data from Balance Sheets.
df2 = df_balance_ttm[TOTAL_EQUITY]

# Join into a single DataFrame.
df_join = pd.concat([df1, df2], axis=1)

# Show the result.
df_join

Net Income  Total Equity
Ticker Report Date                          
A      2010-07-31    417000000  2.810000e+09
       2010-10-31    684000000  3.236000e+09
       2011-01-31    798000000  3.339000e+09
       2011-04-30    890000000  3.961000e+09
       2011-07-31   1015000000  4.200000e+09
...                        ...           ...
low    2018-05-31   3833000000  5.745000e+09
       2018-08-31   3934000000  5.781000e+09
       2018-11-30   3691000000  5.394000e+09
       2019-02-28   2314000000  3.644000e+09
       2019-05-31   2372000000  3.236000e+09

[52672 rows x 2 columns]

In [19]:
# Function for calculating ROE for a single company.
roe = lambda df_grp: df_grp[NET_INCOME] / df_grp[TOTAL_EQUITY].shift(4)

# Split the DataFrame into sub-groups for the tickers,
# then apply the ROE function to each group,
# and finally glue the results back together.
df_roe = df_join.groupby(TICKER, group_keys=False).apply(roe)

# Show the result.
df_roe

Ticker  Report Date
A       2010-07-31          NaN
        2010-10-31          NaN
        2011-01-31          NaN
        2011-04-30          NaN
        2011-07-31     0.361210
                         ...   
low     2018-05-31     0.693003
        2018-08-31     0.710621
        2018-11-30     0.642807
        2019-02-28     0.394006
        2019-05-31     0.412881
Length: 52672, dtype: float64

In [20]:
# Data from Income Statements.
df1 = df_income_ttm[[NET_INCOME, REVENUE]]

# Data from Balance Sheets.
df2 = df_balance_ttm[[TOTAL_ASSETS, TOTAL_EQUITY]]

# Combine the data into a single DataFrame.
df_join = pd.concat([df1, df2], axis=1)

In [21]:
def fin_signals(df):
    """
    Calculate financial signals for a single stock.
    Use sf.apply() with this function for multiple stocks.
    
    :param df:
        Pandas DataFrame with required data from
        Income Statements, Balance Sheets, etc.
        Assumed to be TTM-data.
    
    :return:
        Pandas DataFrame with financial signals.
    """
    
    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=df.index)

    # Net Profit Margin.
    df_signals[NET_PROFIT_MARGIN] = df[NET_INCOME] / df[REVENUE]
    
    # Return on Assets.
    df_signals[ROA] = df[NET_INCOME] / df[TOTAL_ASSETS].shift(4)
    
    # Return on Equity.
    df_signals[ROE] = df[NET_INCOME] / df[TOTAL_EQUITY].shift(4)

    return df_signals

In [22]:
df_fin_signals = sf.apply(df=df_join, func=fin_signals)
df_fin_signals

Net Profit Margin  Return on Assets  Return on Equity
Ticker Report Date                                                       
A      2010-07-31            0.082820               NaN               NaN
       2010-10-31            0.125643               NaN               NaN
       2011-01-31            0.138783               NaN               NaN
       2011-04-30            0.144574               NaN               NaN
       2011-07-31            0.157048          0.111538          0.361210
...                               ...               ...               ...
low    2018-05-31            0.055455          0.101906          0.693003
       2018-08-31            0.055792          0.107287          0.710621
       2018-11-30            0.051871          0.100345          0.642807
       2019-02-28            0.032450          0.065569          0.394006
       2019-05-31            0.033087          0.062034          0.412881

[52672 rows x 3 columns]

In [23]:
# Add time-lag of 30 days.
df_fin_signals2 = sf.add_date_offset(df=df_fin_signals,
                                     date_index=REPORT_DATE,
                                     offset=pd.DateOffset(days=30))

# Show the result.
df_fin_signals2

Net Profit Margin  Return on Assets  Return on Equity
Ticker Report Date                                                       
A      2010-08-30            0.082820               NaN               NaN
       2010-11-30            0.125643               NaN               NaN
       2011-03-02            0.138783               NaN               NaN
       2011-05-30            0.144574               NaN               NaN
       2011-08-30            0.157048          0.111538          0.361210
...                               ...               ...               ...
low    2018-06-30            0.055455          0.101906          0.693003
       2018-09-30            0.055792          0.107287          0.710621
       2018-12-30            0.051871          0.100345          0.642807
       2019-03-30            0.032450          0.065569          0.394006
       2019-06-30            0.033087          0.062034          0.412881

[52672 rows x 3 columns]

In [24]:
# Reindex financial signals to the same days as share-price data.
df_fin_signals_daily = sf.reindex(df_src=df_fin_signals,
                                  df_target=df_prices,
                                  method='ffill')

# Show the result.
df_fin_signals_daily.dropna()

Net Profit Margin  Return on Assets  Return on Equity
Ticker Date                                                             
A      2011-08-01           0.157048          0.111538          0.361210
       2011-08-02           0.157048          0.111538          0.361210
       2011-08-03           0.157048          0.111538          0.361210
       2011-08-04           0.157048          0.111538          0.361210
       2011-08-05           0.157048          0.111538          0.361210
...                              ...               ...               ...
low    2019-07-01           0.033087          0.062034          0.412881
       2019-07-02           0.033087          0.062034          0.412881
       2019-07-03           0.033087          0.062034          0.412881
       2019-07-05           0.033087          0.062034          0.412881
       2019-07-08           0.033087          0.062034          0.412881

[2683482 rows x 3 columns]

In [25]:
df_fin_signals = sf.fin_signals(df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm)

df_fin_signals.dropna().head()

/Users/patrickfahy99/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Report Date                                                      
A      2010-07-31                            0.171492        0.553297   
       2010-10-31                            0.185430        0.561469   
       2011-01-31                            0.365639        0.714818   
       2011-04-30                            0.168927        0.711759   
       2011-07-31                            0.185950        0.738375   

                    CapEx / (Depr + Amor)  Current Ratio  Debt Ratio  \
Ticker Report Date                                                     
A      2010-07-31                0.622857       1.966061    0.404176   
       2010-10-31                0.564356       2.000973    0.380672   
       2011-01-31                0.561947       3.270270    0.265912   
       2011-04-30                0.590551       3.201005    0.247890   
       2011-07-31                0.603113       3.470432    0.247687   

                    Dividends / FCF  Gross Profit Margin  Interest Coverage  \
Ticker Report Date                                                            
A      2010-07-31              -0.0             0.542205           5.636364   
       2010-10-31              -0.0             0.538207           7.447368   
       2011-01-31              -0.0             0.536696           9.106667   
       2011-04-30              -0.0             0.532001          10.890411   
       2011-07-31              -0.0             0.532570          13.927536   

                    Inventory Turnover  Log Revenue  \
Ticker Report Date                                    
A      2010-07-31             7.318314     9.701999   
       2010-10-31             7.603352     9.735918   
       2011-01-31             7.214555     9.759668   
       2011-04-30             7.216882     9.789299   
       2011-07-31             7.205128     9.810434   

                    Net Acquisitions / Total Assets  Net Profit Margin  \
Ticker Report Date                                                       
A      2010-07-31                          0.115385           0.082820   
       2010-10-31                          0.114274           0.125643   
       2011-01-31                          0.136251           0.138783   
       2011-04-30                          0.140132           0.144574   
       2011-07-31                          0.012567           0.157048   

                    Quick Ratio  R&D / Gross Profit  R&D / Revenue  \
Ticker Report Date                                                   
A      2010-07-31      1.065135            0.220879       0.119762   
       2010-10-31      1.141096            0.208874       0.112417   
       2011-01-31      2.483642            0.201555       0.108174   
       2011-04-30      2.445352            0.194504       0.103476   
       2011-07-31      2.669767            0.187391       0.099799   

                    Return on Assets  Return on Equity  \
Ticker Report Date                                       
A      2010-07-31           0.045824          0.148399   
       2010-10-31           0.070545          0.211372   
       2011-01-31           0.099204          0.238994   
       2011-04-30           0.102902          0.224691   
       2011-07-31           0.115960          0.241667   

                    Return on Research Capital  Share Buyback / FCF  
Ticker Report Date                                                   
A      2010-07-31                     4.527363             0.171492  
       2010-10-31                     4.787582             0.185430  
       2011-01-31                     4.961415             0.365639  
       2011-04-30                     5.141287             0.168927  
       2011-07-31                     5.336434             0.185950

In [26]:
df_fin_signals = sf.fin_signals(df_prices=df_prices,
                                df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm,
                                fill_method='ffill')

df_fin_signals.dropna().head()

/Users/patrickfahy99/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2010-08-02                           0.171492        0.553297   
       2010-08-03                           0.171492        0.553297   
       2010-08-04                           0.171492        0.553297   
       2010-08-05                           0.171492        0.553297   
       2010-08-06                           0.171492        0.553297   

                   CapEx / (Depr + Amor)  Current Ratio  Debt Ratio  \
Ticker Date                                                           
A      2010-08-02               0.622857       1.966061    0.404176   
       2010-08-03               0.622857       1.966061    0.404176   
       2010-08-04               0.622857       1.966061    0.404176   
       2010-08-05               0.622857       1.966061    0.404176   
       2010-08-06               0.622857       1.966061    0.404176   

                   Dividends / FCF  Gross Profit Margin  Interest Coverage  \
Ticker Date                                                                  
A      2010-08-02             -0.0             0.542205           5.636364   
       2010-08-03             -0.0             0.542205           5.636364   
       2010-08-04             -0.0             0.542205           5.636364   
       2010-08-05             -0.0             0.542205           5.636364   
       2010-08-06             -0.0             0.542205           5.636364   

                   Inventory Turnover  Log Revenue  \
Ticker Date                                          
A      2010-08-02            7.318314     9.701999   
       2010-08-03            7.318314     9.701999   
       2010-08-04            7.318314     9.701999   
       2010-08-05            7.318314     9.701999   
       2010-08-06            7.318314     9.701999   

                   Net Acquisitions / Total Assets  Net Profit Margin  \
Ticker Date                                                             
A      2010-08-02                         0.115385            0.08282   
       2010-08-03                         0.115385            0.08282   
       2010-08-04                         0.115385            0.08282   
       2010-08-05                         0.115385            0.08282   
       2010-08-06                         0.115385            0.08282   

                   Quick Ratio  R&D / Gross Profit  R&D / Revenue  \
Ticker Date                                                         
A      2010-08-02     1.065135            0.220879       0.119762   
       2010-08-03     1.065135            0.220879       0.119762   
       2010-08-04     1.065135            0.220879       0.119762   
       2010-08-05     1.065135            0.220879       0.119762   
       2010-08-06     1.065135            0.220879       0.119762   

                   Return on Assets  Return on Equity  \
Ticker Date                                             
A      2010-08-02          0.045824          0.148399   
       2010-08-03          0.045824          0.148399   
       2010-08-04          0.045824          0.148399   
       2010-08-05          0.045824          0.148399   
       2010-08-06          0.045824          0.148399   

                   Return on Research Capital  Share Buyback / FCF  
Ticker Date                                                         
A      2010-08-02                    4.527363             0.171492  
       2010-08-03                    4.527363             0.171492  
       2010-08-04                    4.527363             0.171492  
       2010-08-05                    4.527363             0.171492  
       2010-08-06                    4.527363             0.171492

In [27]:
df_fin_signals_2y = sf.fin_signals(df_prices=df_prices,
                                   df_income_ttm=df_income_ttm,
                                   df_balance_ttm=df_balance_ttm,
                                   df_cashflow_ttm=df_cashflow_ttm,
                                   func=sf.avg_ttm_2y,
                                   fill_method='ffill')

df_fin_signals_2y.dropna().head()

/Users/patrickfahy99/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2011-08-01                           0.178721        0.645836   
       2011-08-02                           0.178721        0.645836   
       2011-08-03                           0.178721        0.645836   
       2011-08-04                           0.178721        0.645836   
       2011-08-05                           0.178721        0.645836   

                   CapEx / (Depr + Amor)  Current Ratio  Debt Ratio  \
Ticker Date                                                           
A      2011-08-01               0.612985       2.718246    0.325931   
       2011-08-02               0.612985       2.718246    0.325931   
       2011-08-03               0.612985       2.718246    0.325931   
       2011-08-04               0.612985       2.718246    0.325931   
       2011-08-05               0.612985       2.718246    0.325931   

                   Dividends / FCF  Gross Profit Margin  Interest Coverage  \
Ticker Date                                                                  
A      2011-08-01             -0.0             0.537387            9.78195   
       2011-08-02             -0.0             0.537387            9.78195   
       2011-08-03             -0.0             0.537387            9.78195   
       2011-08-04             -0.0             0.537387            9.78195   
       2011-08-05             -0.0             0.537387            9.78195   

                   Inventory Turnover  Log Revenue  \
Ticker Date                                          
A      2011-08-01            7.261721     9.756217   
       2011-08-02            7.261721     9.756217   
       2011-08-03            7.261721     9.756217   
       2011-08-04            7.261721     9.756217   
       2011-08-05            7.261721     9.756217   

                   Net Acquisitions / Total Assets  Net Profit Margin  \
Ticker Date                                                             
A      2011-08-01                         0.063976           0.119934   
       2011-08-02                         0.063976           0.119934   
       2011-08-03                         0.063976           0.119934   
       2011-08-04                         0.063976           0.119934   
       2011-08-05                         0.063976           0.119934   

                   Quick Ratio  R&D / Gross Profit  R&D / Revenue  \
Ticker Date                                                         
A      2011-08-01     1.867451            0.204135        0.10978   
       2011-08-02     1.867451            0.204135        0.10978   
       2011-08-03     1.867451            0.204135        0.10978   
       2011-08-04     1.867451            0.204135        0.10978   
       2011-08-05     1.867451            0.204135        0.10978   

                   Return on Assets  Return on Equity  \
Ticker Date                                             
A      2011-08-01          0.080892          0.195033   
       2011-08-02          0.080892          0.195033   
       2011-08-03          0.080892          0.195033   
       2011-08-04          0.080892          0.195033   
       2011-08-05          0.080892          0.195033   

                   Return on Research Capital  Share Buyback / FCF  
Ticker Date                                                         
A      2011-08-01                    4.931899             0.178721  
       2011-08-02                    4.931899             0.178721  
       2011-08-03                    4.931899             0.178721  
       2011-08-04                    4.931899             0.178721  
       2011-08-05                    4.931899             0.178721

In [28]:
# Select the data-columns to calculate growth-rates for.
df = df_income_ttm[[REVENUE, NET_INCOME]]

# Dict mapping to the new column-names.
new_names = {REVENUE: SALES_GROWTH,
             NET_INCOME: EARNINGS_GROWTH}

# Calculate the growth-rates.
df_growth = sf.rel_change(df=df, freq='q', quarters=4,
                          future=False, annualized=False,
                          new_names=new_names)

# Show the result.
df_growth.head()

Sales Growth  Earnings Growth
Ticker Report Date                               
A      2010-07-31            NaN              NaN
       2010-10-31            NaN              NaN
       2011-01-31            NaN              NaN
       2011-04-30            NaN              NaN
       2011-07-31       0.283615         1.434053

In [29]:
# Reindex the growth-rates to the same dates as the share-prices.
df_growth_daily = sf.reindex(df_src=df_growth,
                             df_target=df_prices, method='ffill')

# Show the result.
df_growth_daily.dropna().head()

Sales Growth  Earnings Growth
Ticker Date                                     
A      2011-08-01      0.283615         1.434053
       2011-08-02      0.283615         1.434053
       2011-08-03      0.283615         1.434053
       2011-08-04      0.283615         1.434053
       2011-08-05      0.283615         1.434053

In [30]:
df_growth_signals = \
    sf.growth_signals(df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_balance_ttm=df_balance_ttm,
                      df_balance_qrt=df_balance_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt)

df_growth_signals.dropna().head()

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Report Date                                                        
A      2015-01-31       -0.293194          -0.304762          -0.293194   
       2015-04-30       -0.327771          -0.014231          -0.327771   
       2015-07-31       -0.301445          -0.021722          -0.301445   
       2015-10-31       -0.308460           0.031444          -0.308460   
       2016-01-31       -0.028860          -0.023666          -0.028860   

                    Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Report Date                                                              
A      2015-01-31         -0.444000            -0.073529            -0.676923   
       2015-04-30         -0.495159             0.380952            -0.374101   
       2015-07-31         -0.531339             0.275862            -0.244898   
       2015-10-31         -0.269581             0.261261             1.058824   
       2016-01-31          0.100719            -0.135714             0.920635   

                    FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Report Date                                                             
A      2015-01-31    -0.631922       -1.368794       -1.348993      0.261949   
       2015-04-30    -0.743011       -4.346154       -0.364964      0.630751   
       2015-07-31    -0.526882       -0.488506       -4.708333      1.204905   
       2015-10-31    -0.211111        1.415730        0.524823     -0.002470   
       2016-01-31     0.625369       -0.660465       -2.403846     -0.006394   

                    Sales Growth QOQ  Sales Growth YOY  
Ticker Report Date                                      
A      2015-01-31          -0.016299          0.017857  
       2015-04-30          -0.061404         -0.025304  
       2015-07-31           0.052960          0.004955  
       2015-10-31           0.020710         -0.007670  
       2016-01-31          -0.006763          0.001949

In [31]:
df_growth_signals = \
    sf.growth_signals(df_prices=df_prices,
                      df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_balance_ttm=df_balance_ttm,
                      df_balance_qrt=df_balance_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt,
                      fill_method='ffill')

df_growth_signals.dropna().head()

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Date                                                              
A      2015-02-02      -0.293194          -0.304762          -0.293194   
       2015-02-03      -0.293194          -0.304762          -0.293194   
       2015-02-04      -0.293194          -0.304762          -0.293194   
       2015-02-05      -0.293194          -0.304762          -0.293194   
       2015-02-06      -0.293194          -0.304762          -0.293194   

                   Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Date                                                                    
A      2015-02-02           -0.444            -0.073529            -0.676923   
       2015-02-03           -0.444            -0.073529            -0.676923   
       2015-02-04           -0.444            -0.073529            -0.676923   
       2015-02-05           -0.444            -0.073529            -0.676923   
       2015-02-06           -0.444            -0.073529            -0.676923   

                   FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Date                                                                   
A      2015-02-02   -0.631922       -1.368794       -1.348993      0.261949   
       2015-02-03   -0.631922       -1.368794       -1.348993      0.261949   
       2015-02-04   -0.631922       -1.368794       -1.348993      0.261949   
       2015-02-05   -0.631922       -1.368794       -1.348993      0.261949   
       2015-02-06   -0.631922       -1.368794       -1.348993      0.261949   

                   Sales Growth QOQ  Sales Growth YOY  
Ticker Date                                            
A      2015-02-02         -0.016299          0.017857  
       2015-02-03         -0.016299          0.017857  
       2015-02-04         -0.016299          0.017857  
       2015-02-05         -0.016299          0.017857  
       2015-02-06         -0.016299          0.017857

In [32]:
df_growth_signals_2y = \
    sf.growth_signals(df_prices=df_prices,
                      df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_balance_ttm=df_balance_ttm,
                      df_balance_qrt=df_balance_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt,
                      fill_method='ffill',
                      func=sf.avg_ttm_2y)

df_growth_signals_2y.dropna().head()

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Date                                                              
A      2016-02-01      -0.161027          -0.164214          -0.161027   
       2016-02-02      -0.161027          -0.164214          -0.161027   
       2016-02-03      -0.161027          -0.164214          -0.161027   
       2016-02-04      -0.161027          -0.164214          -0.161027   
       2016-02-05      -0.161027          -0.164214          -0.161027   

                   Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Date                                                                    
A      2016-02-01         -0.17164            -0.104622             0.121856   
       2016-02-02         -0.17164            -0.104622             0.121856   
       2016-02-03         -0.17164            -0.104622             0.121856   
       2016-02-04         -0.17164            -0.104622             0.121856   
       2016-02-05         -0.17164            -0.104622             0.121856   

                   FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Date                                                                   
A      2016-02-01   -0.003277        -1.01463        -1.87642      0.127777   
       2016-02-02   -0.003277        -1.01463        -1.87642      0.127777   
       2016-02-03   -0.003277        -1.01463        -1.87642      0.127777   
       2016-02-04   -0.003277        -1.01463        -1.87642      0.127777   
       2016-02-05   -0.003277        -1.01463        -1.87642      0.127777   

                   Sales Growth QOQ  Sales Growth YOY  
Ticker Date                                            
A      2016-02-01         -0.011531          0.009903  
       2016-02-02         -0.011531          0.009903  
       2016-02-03         -0.011531          0.009903  
       2016-02-04         -0.011531          0.009903  
       2016-02-05         -0.011531          0.009903

In [33]:
df_sales_per_share = df_income_ttm[REVENUE].div(df_income_ttm[SHARES_DILUTED], axis=0)
df_sales_per_share

Ticker  Report Date
A       2010-07-31     14.354954
        2010-10-31     15.378531
        2011-01-31     16.231475
        2011-04-30     17.365303
        2011-07-31     18.167252
                         ...    
low     2018-05-31     83.075721
        2018-08-31     85.469091
        2018-11-30     86.909313
        2019-02-28     87.818966
        2019-05-31     89.083566
Length: 52672, dtype: float64

In [34]:
df_sps_daily = sf.reindex(df_src=df_sales_per_share,
                          df_target=df_prices, method='ffill')

df_sps_daily.dropna()

Ticker  Date      
A       2010-08-02    14.354954
        2010-08-03    14.354954
        2010-08-04    14.354954
        2010-08-05    14.354954
        2010-08-06    14.354954
                        ...    
low     2019-07-01    89.083566
        2019-07-02    89.083566
        2019-07-03    89.083566
        2019-07-05    89.083566
        2019-07-08    89.083566
Length: 3106715, dtype: float64

In [35]:
# Calculate P/Sales ratio.
df_psales = df_prices[CLOSE].div(df_sps_daily, axis=0)

# Rename the data-column.
df_psales.rename(PSALES, inplace=True)

# Show the result.
df_psales.dropna()

Ticker  Date      
A       2010-08-02    2.007669
        2010-08-03    1.939400
        2010-08-04    1.970748
        2010-08-05    1.982591
        2010-08-06    2.001400
                        ...   
low     2019-07-01    1.147237
        2019-07-02    1.152738
        2019-07-03    1.157902
        2019-07-05    1.169913
        2019-07-08    1.174852
Name: P/Sales, Length: 3102739, dtype: float64

In [36]:
df_earnings_per_share = df_income_ttm[NET_INCOME_COMMON].div(df_income_ttm[SHARES_DILUTED], axis=0)
df_earnings_per_share

Ticker  Report Date
A       2010-07-31     1.188881
        2010-10-31     1.932203
        2011-01-31     2.252646
        2011-04-30     2.510578
        2011-07-31     2.853127
                         ...   
low     2018-05-31     4.606971
        2018-08-31     4.768485
        2018-11-30     4.508092
        2019-02-28     2.849754
        2019-05-31     2.947499
Length: 52672, dtype: float64

In [37]:
df_eps_daily = sf.reindex(df_src=df_earnings_per_share,
                          df_target=df_prices, method='ffill')

df_eps_daily.dropna()

Ticker  Date      
A       2010-08-02    1.188881
        2010-08-03    1.188881
        2010-08-04    1.188881
        2010-08-05    1.188881
        2010-08-06    1.188881
                        ...   
low     2019-07-01    2.947499
        2019-07-02    2.947499
        2019-07-03    2.947499
        2019-07-05    2.947499
        2019-07-08    2.947499
Length: 3155274, dtype: float64

In [38]:
# Calculate the P/E ratio.
df_pe = df_prices[CLOSE] / df_eps_daily

# Rename the data-column.
df_pe.rename(PE, inplace=True)

# Show the result.
df_pe.dropna()

Ticker  Date      
A       2010-08-02    24.241283
        2010-08-03    23.416978
        2010-08-04    23.795486
        2010-08-05    23.938477
        2010-08-06    24.165582
                        ...    
low     2019-07-01    34.673461
        2019-07-02    34.839704
        2019-07-03    34.995768
        2019-07-05    35.358788
        2019-07-08    35.508067
Name: P/E, Length: 3151265, dtype: float64

In [39]:
# Calculate Free Cash Flow.
df_fcf_ttm = df_cashflow_ttm[NET_CASH_OPS] + df_cashflow_ttm[CAPEX]

# Rename the data-column.
df_fcf_ttm.rename(FCF, inplace=True)

Ticker  Report Date
A       2010-07-31     4.490000e+08
        2010-10-31     6.040000e+08
        2011-01-31     6.810000e+08
        2011-04-30     8.110000e+08
        2011-07-31     9.680000e+08
                           ...     
low     2018-05-31     4.098000e+09
        2018-08-31     4.653000e+09
        2018-11-30     5.383000e+09
        2019-02-28     5.095000e+09
        2019-05-31     3.841000e+09
Name: Free Cash Flow, Length: 52672, dtype: float64

In [40]:
df_fcf_per_share = df_fcf_ttm.div(df_income_ttm[SHARES_DILUTED])
df_fcf_per_share

Ticker  Report Date
A       2010-07-31     1.280114
        2010-10-31     1.706215
        2011-01-31     1.922371
        2011-04-30     2.287729
        2011-07-31     2.721012
                         ...   
low     2018-05-31     4.925481
        2018-08-31     5.640000
        2018-11-30     6.574656
        2019-02-28     6.274631
        2019-05-31     4.772911
Length: 52672, dtype: float64

In [41]:
df_fcf_daily = sf.reindex(df_src=df_fcf_per_share,
                          df_target=df_prices, method='ffill')

df_fcf_daily.dropna()

Ticker  Date      
A       2010-08-02    1.280114
        2010-08-03    1.280114
        2010-08-04    1.280114
        2010-08-05    1.280114
        2010-08-06    1.280114
                        ...   
low     2019-07-01    4.772911
        2019-07-02    4.772911
        2019-07-03    4.772911
        2019-07-05    4.772911
        2019-07-08    4.772911
Length: 3095210, dtype: float64

In [42]:
# Calculate the P/FCF ratio.
df_pfcf = df_prices[CLOSE] / df_fcf_daily

# Rename the data-column.
df_pfcf.rename(PFCF, inplace=True)

# Show the result.
df_pfcf.dropna()

Ticker  Date      
A       2010-08-02    22.513619
        2010-08-03    21.748062
        2010-08-04    22.099594
        2010-08-05    22.232394
        2010-08-06    22.443313
                        ...    
low     2019-07-01    21.412510
        2019-07-02    21.515172
        2019-07-03    21.611550
        2019-07-05    21.835732
        2019-07-08    21.927919
Name: P/FCF, Length: 3092855, dtype: float64

In [43]:
avg_ttm_2y = lambda df: 0.5 * (df + df.shift(4))

In [44]:
df_earnings_2y = sf.apply(df=df_income_ttm[NET_INCOME_COMMON],
                          func=avg_ttm_2y)

df_earnings_2y

Ticker  Report Date
A       2010-07-31              NaN
        2010-10-31              NaN
        2011-01-31              NaN
        2011-04-30              NaN
        2011-07-31     7.160000e+08
                           ...     
low     2018-05-31     3.322000e+09
        2018-08-31     3.498500e+09
        2018-11-30     3.623500e+09
        2019-02-28     2.880500e+09
        2019-05-31     3.102500e+09
Name: Net Income (Common), Length: 52672, dtype: float64

In [45]:
df_eps_2y = df_earnings_2y.div(df_income_ttm[SHARES_DILUTED])
df_eps_2y.dropna()

Ticker  Report Date
A       2011-07-31     2.012649
        2011-10-31     2.388732
        2012-01-31     2.606916
        2012-04-30     2.816384
        2012-07-31     2.878187
                         ...   
low     2018-05-31     3.992788
        2018-08-31     4.240606
        2018-11-30     4.425649
        2019-02-28     3.547414
        2019-05-31     3.855235
Length: 44777, dtype: float64

In [46]:
df_eps_2y_daily = sf.reindex(df_src=df_eps_2y,
                             df_target=df_prices, method='ffill')

df_eps_2y_daily.dropna()

Ticker  Date      
A       2011-08-01    2.012649
        2011-08-02    2.012649
        2011-08-03    2.012649
        2011-08-04    2.012649
        2011-08-05    2.012649
                        ...   
low     2019-07-01    3.855235
        2019-07-02    3.855235
        2019-07-03    3.855235
        2019-07-05    3.855235
        2019-07-08    3.855235
Length: 2698973, dtype: float64

In [47]:
# Calculate the P/E ratios using 2-year earnings average.
df_pe_2y = df_prices[CLOSE].div(df_eps_2y_daily, axis=0)

# Rename the data-column.
PE_2Y = 'P/E (2Y Avg. Earnings)'
df_pe_2y.rename(PE_2Y, inplace=True)

# Show the result.
df_pe_2y.dropna()

Ticker  Date      
A       2011-08-01    20.266819
        2011-08-02    19.347633
        2011-08-03    19.476816
        2011-08-04    18.339012
        2011-08-05    18.180017
                        ...    
low     2019-07-01    26.509412
        2019-07-02    26.636512
        2019-07-03    26.755830
        2019-07-05    27.033375
        2019-07-08    27.147505
Name: P/E (2Y Avg. Earnings), Length: 2696195, dtype: float64

In [48]:
def val_signals(df_prices, df_income_ttm, df_cashflow_ttm,
                shares_index=SHARES_DILUTED):
    """
    Calculate valuation signals for all stocks in the DataFrames.
    
    :param df_prices:
        Pandas DataFrame with share-prices for multiple stocks.
        
    :param df_income_ttm:
        Pandas DataFrame with Income Statement TTM data for
        multiple stocks.
    
    :param df_cashflow_ttm:
        Pandas DataFrame with Cash-Flow Statement TTM data for
        multiple stocks.

    :param shares_index:
        String with the data-column name for the share-count
        e.g. SHARES_DILUTED or SHARES_BASIC.
    
    :return:
        Pandas DataFrame with valuation signals.
    """

    # Create a DataFrame with the financial data we need.
    # Start by copying data from the Income Statements.
    df = df_income_ttm[[REVENUE, NET_INCOME_COMMON]].copy()

    # Calculate FCF and add it as a new column to the DataFrame.
    df[FCF] = df_cashflow_ttm[NET_CASH_OPS] + df_cashflow_ttm[CAPEX]

    # Calculate Per-Share numbers.
    df_per_share = df.div(df_income_ttm[shares_index], axis=0)
    
    # Reindex the per-share financial data to daily data-points.
    df_daily = sf.reindex(df_src=df_per_share,
                          df_target=df_prices,
                          method='ffill')
    
    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=df_prices.index)
    
    # Use the closing share-price for all these signals.
    df_price = df_prices[CLOSE]
    
    # P/Sales ratio.
    df_signals[PSALES] = df_price / df_daily[REVENUE]
    
    # P/E ratio.
    df_signals[PE] = df_price / df_daily[NET_INCOME_COMMON]

    # P/FCF ratio.
    df_signals[PFCF] = df_price / df_daily[FCF]

    return df_signals

In [49]:
df_val_signals = val_signals(df_prices=df_prices,
                             df_income_ttm=df_income_ttm,
                             df_cashflow_ttm=df_cashflow_ttm,
                             shares_index=SHARES_DILUTED)

df_val_signals.dropna()

P/Sales        P/E      P/FCF
Ticker Date                                      
A      2010-08-02  2.007669  24.241283  22.513619
       2010-08-03  1.939400  23.416978  21.748062
       2010-08-04  1.970748  23.795486  22.099594
       2010-08-05  1.982591  23.938477  22.232394
       2010-08-06  2.001400  24.165582  22.443313
...                     ...        ...        ...
low    2019-07-01  1.147237  34.673461  21.412510
       2019-07-02  1.152738  34.839704  21.515172
       2019-07-03  1.157902  34.995768  21.611550
       2019-07-05  1.169913  35.358788  21.835732
       2019-07-08  1.174852  35.508067  21.927919

[3055286 rows x 3 columns]

In [50]:
df_val_signals = sf.val_signals(df_prices=df_prices,
                                df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm)

In [51]:
df_val_signals.dropna().head()

Dividend Yield  Earnings Yield  FCF Yield    Market-Cap  \
Ticker Date                                                                  
A      2012-04-30        0.002344        0.073937   0.073736  1.493172e+10   
       2012-05-01        0.002301        0.072577   0.072380  1.521138e+10   
       2012-05-02        0.002297        0.072459   0.072262  1.523616e+10   
       2012-05-03        0.002342        0.073884   0.073683  1.494234e+10   
       2012-05-04        0.002411        0.076046   0.075839  1.451754e+10   

                     P/Book    P/Cash        P/E      P/FCF     P/NCAV  \
Ticker Date                                                              
A      2012-04-30  3.160152  3.832577  13.525109  13.561962  11.294796   
       2012-05-01  3.219340  3.904358  13.778424  13.815967  11.506339   
       2012-05-02  3.224584  3.910719  13.800870  13.838474  11.525083   
       2012-05-03  3.162400  3.835303  13.534728  13.571608  11.302829   
       2012-05-04  3.072495  3.726268  13.149946  13.185777  10.981498   

                    P/NetNet   P/Sales  
Ticker Date                             
A      2012-04-30  41.105905  2.200047  
       2012-05-01  41.875788  2.241252  
       2012-05-02  41.944006  2.244903  
       2012-05-03  41.135141  2.201612  
       2012-05-04  39.965699  2.139022

In [52]:
df_val_signals_3y = sf.val_signals(df_prices=df_prices,
                                   df_income_ttm=df_income_ttm,
                                   df_balance_ttm=df_balance_ttm,
                                   df_cashflow_ttm=df_cashflow_ttm,
                                   func=sf.avg_ttm_3y)

In [53]:
df_val_signals.describe()

,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
count,1.552937e+06,3.151393e+06,3.151393e+06,3.151393e+06,3.151393e+06,3.146700e+06,3.151393e+06,3.151393e+06,3.151393e+06,3.151393e+06,3.102867e+06
mean,2.903928e-02,-3.544072e-01,5.232956e-01,1.264657e+10,inf,inf,inf,inf,inf,2.560581e+02,inf
std,6.888628e-02,6.311345e+01,1.274236e+02,4.313589e+10,NaN,NaN,NaN,NaN,NaN,7.216269e+04,NaN
min,-3.656770e-01,-1.640625e+04,-1.464086e+02,3.679200e+03,-8.370529e+04,4.173913e-05,-5.749890e+05,-6.283370e+05,-1.396542e+06,-1.231029e+06,1.350923e-06
25%,1.067073e-02,-3.645839e-03,-1.586495e-03,4.568253e+08,1.420934e+00,5.593932e+00,-6.876605e-01,-6.973385e-01,-4.239573e+00,-4.607990e+00,8.150961e-01
50%,1.950426e-02,3.597931e-02,3.834221e-02,2.457343e+09,2.569171e+00,1.232623e+01,1.682580e+01,1.437207e+01,-9.743329e-01,-1.578039e+00,1.735835e+00
75%,3.201077e-02,5.804814e-02,6.844751e-02,9.203428e+09,4.682983e+00,3.210008e+01,2.697987e+01,2.561564e+01,5.036982e+00,1.955413e+00,3.674149e+00
max,4.673341e+00,4.470199e+01,3.313802e+04,1.296584e+12,inf,inf,inf,inf,inf,2.471105e+07,inf


In [54]:
%%time
# Combine the DataFrames.
dfs = [df_fin_signals, df_growth_signals, df_val_signals]
df_signals = pd.concat(dfs, axis=1)

CPU times: user 735 ms, sys: 1.91 s, total: 2.64 s
Wall time: 3.52 s


In [ ]:
# Remove all rows with only NaN values.
df = df_signals.dropna().reset_index(drop=True)

# For each column, show the fraction of the rows that are NaN.
(df.isnull().sum() / len(df)).sort_values(ascending=False)

In [ ]:
# List of the columns before removing any.
columns_before = df_signals.columns

# Threshold for the number of rows that must be NaN for each column.
thresh = 0.75 * len(df_signals.dropna(how='all'))

# Remove all columns which don't have sufficient data.
df_signals = df_signals.dropna(axis='columns', thresh=thresh)

# List of the columns after the removal.
columns_after = df_signals.columns

# Show the columns that were removed.
columns_before.difference(columns_after)

In [ ]:
# Daily Share-Prices.
df_prices = sf.load_shareprices(variant='daily', market=market)
df_prices = df_prices.loc[tickers, [CLOSE, ADJ_CLOSE]].copy()

In [ ]:
sf.rel_change(df=df_prices, freq='bdays',
              weeks=1, months=2, years=3,
              future=False, annualized=True).dropna()

In [ ]:
TOTAL_RETURN_1_3Y = 'Mean-Log Total Return 1-3 Years'

In [ ]:
# Dict mapping old to new column-names.
new_names_1_3y = {SHARE_PRICE: SHARE_PRICE_1_3Y,
                  TOTAL_RETURN: TOTAL_RETURN_1_3Y}

In [ ]:
# Calculate the future mean-log annualized 1-3 year changes.
df_mean_chg = sf.mean_log_change(df=df_prices, freq='bdays',
                                 future=True, annualized=True,
                                 min_years=1, max_years=3,
                                 new_names=new_names_1_3y)

In [ ]:
%%time
# Calculate the mean log-returns for all 1-3 year periods.
df_returns_1_3y = \
    hub.mean_log_returns(name=TOTAL_RETURN_1_3Y,
                         future=True, annualized=True,
                         min_years=1, max_years=3)

In [ ]:
# Combine the signals and stock-returns.
# We are only using the rows which are NetNet discounts.
dfs = [df_signals, df_returns_1_3y]
df_sig_rets = pd.concat(dfs, axis=1)

In [ ]:
# Calculate the correlation between all signals and stock-returns.
df_corr = df_sig_rets.corr()

# Show how the signals are correlated with the stock-returns.
# Sorted to show the strongest absolute correlations first.
df_corr_returns = df_corr[TOTAL_RETURN_1_3Y].abs().sort_values(ascending=False)
df_corr_returns

In [ ]:
# Only show the large (absolute) correlation values.
df_corr2 = df_corr[df_corr.abs() > 0.7]

# Round correlation values to 2 digits.
df_corr2 = df_corr2.round(2)

# Transform the table to give a better overview.
df_corr2 = df_corr2.stack()

# Remove all values that are 1.0
df_corr2 = df_corr2[df_corr2 != 1.0]

# Show the result. Use a DataFrame for pretty printing.
pd.DataFrame(df_corr2, columns=['Correlation'])

In [ ]:
def regression(df, use_constant=True):
    """
    Perform multiple linear-regression on the given data.
    
    :param df:
        Pandas DataFrame with signals and returns.
        
    :param use_constant:
        Boolean whether to add a 'Constant' column to
        find the bias.
    
    :return:
        StatsModels Regression Results.
    """
    
    # Remove rows where all values are missing.
    df = df.dropna(how='any')

    # DataFrame which only contains the signals.
    df_x = df.drop(columns=[TOTAL_RETURN_1_3Y])
    
    # DataFrame which only contains the stock-returns.
    df_y = df[TOTAL_RETURN_1_3Y]

    # Standardize the signals so they have mean 0 and std 1.
    df_x = (df_x - df_x.mean()) / df_x.std()

    # Add a "constant" column so the regression can find the bias.
    if use_constant:
        df_x['Constant'] = 1.0

    # Perform the regression on this data.
    model = sm.OLS(df_y, df_x).fit()
    
    return model

In [ ]:
# Only use some of the signals.
columns = [TOTAL_RETURN_1_3Y, GROSS_PROFIT_MARGIN, SALES_GROWTH,
           CURRENT_RATIO, LOG_REVENUE, P_NETNET, P_CASH,
           MARKET_CAP, SALES_GROWTH_YOY, ASSETS_GROWTH_QOQ,
           NET_PROFIT_MARGIN, INTEREST_COVERAGE]
df = df_sig_rets[columns]


In [ ]:
# Perform the Linear Regression on the signals and stock-returns.
model = regression(df=df)

# Show the results.
model.summary()